In [76]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import os
import importlib
import download_clean_data as dc
import ipca
import metrics
import kernel_regression as kr
import cross_validaton as cv
importlib.reload(dc) 
importlib.reload(ipca)
importlib.reload(metrics)
importlib.reload(kr)
importlib.reload(cv)

<module 'cross_validaton' from 'c:\\Users\\dandr_wh38grr\\Documents\\GitHub\\ML-project-2\\cross_validaton.py'>

In [73]:
folder_path = "Data/monthly_data"

In [74]:
N = 100
date = 20150000
data, ret = dc.download_clean_data(folder_path, date, N)

In [77]:
k = 5
gamma_first = ipca.gamma_first(ret, data, k)    

In [78]:
max_iter = 10
#gamma, f_list = ipca.ipca(data, ret, gamma_first.copy(), max_iter)
W = np.eye(N)
W_list = [W]*(len(data))
lambda1 = 0.1
lambda2 = 0.1
gamma_reg_w, f_list_reg_w = ipca.ipca_reg_w(data, ret, gamma_first.copy(), max_iter, lambda1, lambda2, W_list)

In [6]:
print(metrics.total_R_squared(ret, data, gamma, f_list),metrics.pred_R_squared(ret, data, gamma, f_list))

0.26979192673705055 0.005220343383721682


In [79]:
print(metrics.total_R_squared(ret, data, gamma_reg_w, f_list_reg_w),metrics.pred_R_squared(ret, data, gamma_reg_w, f_list_reg_w))

0.25135949398526714 0.013107183904452202


In [60]:
y = ret
x = data
trsh = 0.8
lambda1_v = list([1])
lambda2_v = list([1])
max_iter = 3

cv.cross_val_IPCA_reg(y,x, trsh, lambda1_v, lambda2_v, gamma_first, max_iter, W_list)

({('IPCA_reg', 1, 1): 0.147129208203027},
 {('IPCA_reg', 1, 1): 0.048385453820912194})

In [9]:
cv.cross_val_IPCA(y,x, 0.9, gamma_first, max_iter)

done


({'IPCA': 0.12064823317073448}, {'IPCA': 0.01291255112364964})

In [61]:
data2 = data.copy()
data2 = np.array(np.array(data2).reshape(72*100,94)) #flatten data, build X
tk = 1
l = 10
K = kr.K_LR(data2, tk, l)

In [68]:
_, f_list = ipca.ipca(data, ret, gamma_first.copy(), 1)

lambda1 = 1
lambda2 = 1

Omega1=np.eye((len(data))*N)
Omega2=np.eye(N)
f_list_kr, v_kr, Q_kr, _, _ = kr.kernel_regression(data, ret, f_list.copy(), lambda1, lambda2, Omega1, Omega2, 1, N, K)

0


In [65]:
print(metrics.total_R_squared_kr(ret, v_kr, Q_kr))

0.4647376938059905


In [69]:
print(metrics.total_R_squared_kr(ret, v_kr, Q_kr))

0.037003227875861855


In [71]:
_, f_list = ipca.ipca(data, ret, gamma_first.copy(), 1)

lambda1_v = list([1])
lambda2_v = list([1])
alphas_v = list([10])
cv.cross_val_gaussian(y,x, trsh, lambda1_v, lambda2_v, alphas_v, N, f_list.copy(), Omega2, 3)

start cycle
0
1
2
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done


{('Gaussian', 1, 1, 10): array([0.4862322])}

In [ ]:
0.4119

In [27]:
lambda1_v = list([0.1])
lambda2_v = list([0.1])

_, f_list = ipca.ipca(data, ret, gamma_first.copy(), 1)

cv.cross_val_linear(y,x, trsh, lambda1_v, lambda2_v, N, f_list.copy(), Omega2, 30)

start cycle
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done


{('Linear', 0.1, 0.1): array([0.12346826])}

### optional

In [ ]:
m_hat = 10
L, B = kr.pivoted_chol(K, m_hat)

In [ ]:
np.linalg.norm(L@L.T - K)

2046.1921825040288

In [ ]:
f_list_kr_LR, v_kr_LR = kr.kernel_regression_LR(data, K, B, ret, f_list.copy(), lambda1, lambda2, Omega2, 10, m_hat, N)

0
1
2
3
4
5
6
7
8
9


In [ ]:
print(metrics.total_R_squared_kr_LR(ret, B, K, v_kr_LR, f_list_kr_LR))

0.0017402896001241785


In [ ]:
alphas_v = list([1])
cv.cross_val_gaussian(y,x, 0.8, lambda1_v, lambda2_v, alphas_v, N, f_list, Omega2, 10)

0
1
2
3
4
5
6
7
8
9
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done1
done


{('Gaussian', 0.1, 0.1, 1): 0.6073464811374132}

In [ ]:
cv.cross_val_gaussian_LR(y, x, 0.8, lambda1_v, lambda2_v, alphas_v, N, f_list, Omega2, 10, m_hat)


1
2
3
4
5
6
7
8
9
1
2
3
4
5
6
7
8
9
39.86480749253068
0
1
2
3
4
5
6
7
8
9
0.001


{('Gaussian', 0.1, 0.1, 1): 0.0}

In [ ]:
def cross_val_linear(y,x, trsh, lambda1_v, lambda2_v, N, f_list_input, Omega2, max_iter):

    total_R2_dict = {}
    
    xx_train,yy_train,xx_test,yy_test = split_dataset(x,y, trsh)
    Omega_test = np.eye(len(xx_test)*N)
    Omega_train = np.eye(len(xx_train)*N)

    data2_train = xx_train.copy()
    data2_train = np.array(np.array(data2_train).reshape(len(xx_train)*N,94)) #flatten data, build X
    data2_test = xx_test.copy()
    data2_test = np.array(np.array(data2_test).reshape(len(xx_test)*N,94))
    K_train = kr.K_LR(data2_train, 0, 1)
    K_test_train = kr.Kernel(data2_test, data2_train, 0, 1)
    K_test = kr.K_LR(data2_test, 0, 1)
    print('start cycle')

    g_list = []
    

    for lambda1 in lambda1_v:
        for lambda2 in lambda2_v:
            f_list, v, _, _, _ = kr.kernel_regression(xx_train, yy_train, f_list_input.copy(), lambda1, lambda2, Omega_train, Omega2, max_iter, N, K_train)

            yy_pred = []

            g_list = []
            c_list = []
            for t in range(0,len(xx_test)):

                g = kr.solve_g_kernel(xx_train, f_list, v, t, K_test_train)
                G = g@g.T

                g_list.append(g)

                c = np.linalg.solve(G+lambda2*Omega2, yy_test[t])

                c_list.append(c)

                print('done1')
                yy_pred.append(g.T@c)

            print('done')

            #vt, Qt = kr.solve_v_kernel(yy_test, lambda1, xx_test, yy_pred, N, Omega_test, K_test)

            total_R2_dict[('Linear', lambda1, lambda2)] = metrics.total_R_squared_kr_out_os(yy_test, g_list, c_list)
                
    return total_R2_dict